In [1]:
import torch
import numpy as np

In [2]:
batch = 4
seq_len = 2
dim = 2
a=torch.rand(batch,seq_len,dim)
b=torch.rand(batch,1,dim)
# b = b.repeat(1,seq_len,1)
# res = torch.concat([a,b],dim=-1)
res = a+b
print(a,b,res)
print(res.shape)

tensor([[[0.6230, 0.4881],
         [0.4676, 0.6695]],

        [[0.3570, 0.1459],
         [0.8052, 0.2505]],

        [[0.7765, 0.9509],
         [0.6656, 0.2447]],

        [[0.7598, 0.0377],
         [0.7285, 0.6509]]]) tensor([[[0.8809, 0.7215]],

        [[0.0021, 0.5070]],

        [[0.4040, 0.8423]],

        [[0.9786, 0.6530]]]) tensor([[[1.5039, 1.2096],
         [1.3485, 1.3910]],

        [[0.3590, 0.6529],
         [0.8072, 0.7575]],

        [[1.1805, 1.7932],
         [1.0696, 1.0869]],

        [[1.7384, 0.6907],
         [1.7071, 1.3040]]])
torch.Size([4, 2, 2])


In [3]:
output = torch.rand(4,5)
target = torch.randint(0,5,(4,1))
pred = output.topk(5, 1, True, True)[1].t()
expand_target = target.view(1, -1).expand_as(pred)
correct = pred.eq(expand_target)

In [4]:
output = torch.rand(4,5)
top1 = output.topk(1)[1]
print(top1[0])

tensor([0])


In [5]:
import random
res = random.sample(range(1000),10)
print(res)

[444, 174, 922, 728, 512, 68, 784, 562, 278, 812]


In [6]:
def sapmle_topk1_prototypes(logits:torch.Tensor, targets:torch.Tensor, topK:int):
    cls_num = logits.shape[1]
    batch = logits.shape[0]
    

    origin_target = []
    new_target = torch.zeros(batch,1).view(-1)
    for i, target in enumerate(targets):
        random_targets = random.sample(range(cls_num),topK+1)
        if target in random_targets:
            new_target[i] = random_targets.index(target)
        else:
            random_targets[0] = target.numpy()
        origin_target =  np.append(origin_target, random_targets)
    
    origin_target = origin_target.reshape(batch,-1)
    new_target = new_target.cuda()

    return new_target, origin_target

In [7]:
cls_num = 10
batch = 4
logits = torch.rand(batch,cls_num)
target = torch.randint(0,cls_num,(batch,1)).view(-1)
new_target, origin_target = sapmle_topk1_prototypes(logits,target,5)

print(logits.topk(5)[1])
print(target)
print(new_target)
print(origin_target)

tensor([[4, 1, 2, 9, 5],
        [5, 0, 6, 4, 7],
        [1, 7, 0, 8, 4],
        [8, 0, 4, 2, 9]])
tensor([7, 4, 1, 8])
tensor([0., 1., 5., 0.], device='cuda:0')
[[7. 3. 2. 0. 5. 1.]
 [3. 4. 0. 6. 8. 9.]
 [3. 6. 4. 7. 9. 1.]
 [8. 4. 2. 9. 6. 0.]]


# .numpy()

In [8]:
a = torch.tensor([1,2])
print(a)
b = a.numpy()
print(b)

tensor([1, 2])
[1 2]


# sample

In [9]:
random.seed(1)
for i in range(2):
    for j in range(2):
        res = random.sample(range(10),5)
        print(res)

[2, 1, 4, 0, 3]
[7, 9, 6, 8, 1]
[1, 7, 0, 6, 3]
[6, 0, 7, 2, 5]


# att rank

In [10]:
batch = 1
k = 2
y = abs(torch.rand(batch, k))
res = abs(torch.rand(batch, k))
print(y)
print(res)

tensor([[0.5577, 0.8416]])
tensor([[0.6832, 0.0338]])


In [11]:
def att_rank(y):
    exp_y = torch.exp(y)
    # print(f'exp_y:{exp_y}')
    sum = torch.sum(exp_y,dim=-1).unsqueeze(-1)
    # print(f'sum:{sum}')
    y = exp_y / sum
    return y

In [12]:
y = att_rank(y)
res = att_rank(res)
loss1 = y*torch.log(res)
loss2 = (1-y)*torch.log(1-res)
Loss = loss1+loss2
print(Loss.sum())

tensor(-1.5815)


# relu

In [13]:
import torch.nn.functional as F

x = torch.tensor([-1,1])
print(x)
print(F.relu(x))

tensor([-1,  1])
tensor([0, 1])


In [14]:
cls_num = 10
batch = 4
k = 5
logits = torch.rand(batch,cls_num)
print(f'logits:{logits}')
indices = torch.randint(0,cls_num,(batch,k))
print(f'indices:{indices}')
logits2 = torch.stack([logits[i][indices[i]] for i in range(batch)])
print(logits2)

logits:tensor([[0.9215, 0.6952, 0.4808, 0.0403, 0.5731, 0.2616, 0.7758, 0.7189, 0.1369,
         0.4465],
        [0.1600, 0.2066, 0.4691, 0.8494, 0.7783, 0.4414, 0.2450, 0.7125, 0.5073,
         0.7279],
        [0.1275, 0.2685, 0.6627, 0.8851, 0.8482, 0.3214, 0.1433, 0.9075, 0.5433,
         0.4391],
        [0.6803, 0.1889, 0.5704, 0.9182, 0.1282, 0.7901, 0.9930, 0.7007, 0.3384,
         0.6612]])
indices:tensor([[4, 8, 2, 0, 1],
        [7, 6, 3, 3, 7],
        [0, 7, 4, 3, 2],
        [9, 4, 7, 1, 6]])
tensor([[0.5731, 0.1369, 0.4808, 0.9215, 0.6952],
        [0.7125, 0.2450, 0.8494, 0.8494, 0.7125],
        [0.1275, 0.9075, 0.8482, 0.8851, 0.6627],
        [0.6612, 0.1282, 0.7007, 0.1889, 0.9930]])


In [15]:
def one_hot(y, cls_num):
    one_hot_target = torch.zeros((y.shape[0], cls_num))
    for i,target in enumerate(y):
            one_hot_target[i][target] = 1
    return one_hot_target

In [16]:
a = torch.tensor([1,2,3])
res = one_hot(a,100)
print(res)

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         

In [71]:
a = torch.rand(1,5)
b = torch.rand(1,5)
res1 = a @ b.T
a = F.normalize(a,dim=-1)
b = F.normalize(b,dim=-1)
res2 = a @ b.T

# print(a)
# print(b)
# print(res1)
# print(res2)